## NLP 
I use AWS to performe my NLP processing

In [1]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO

In [2]:
sys.path.insert(1, '../src')
from NLP import nlp_process
from Content_recommender import ContentRecommender

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload

In [5]:
s3 = boto3.client('s3')

#### For AWS

In [7]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

#### For local machine

In [11]:
df_all_15 = pd.read_csv("../../Data_recommender/df_all_15_text.csv")

In [12]:
df_all_15.shape

(370252, 21)

In [13]:
df_all_15.head(5)

,Unnamed: 0,category,description,title,also_buy,image_x,brand,feature,rank,also_view,...,price,asin,overall,verified,reviewText,summary,unixReviewTime,style,item_text,title_nlp
0,0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",...,$19.95,0899332757,5.0,False,your interest explor area new york beyond metr...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'},garmingreat explor rural part stateformat pape...,delorm new york state atla amp gazett
1,1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",...,$19.95,0899332757,4.0,True,wonder comprehens atla littl larg,Four Stars,1438646400,{'Format:': ' Paperback'},garminfour starsformat paperback,delorm new york state atla amp gazett
2,2,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",...,$19.95,0899332757,5.0,True,great product fast deliveri need rv travel,Super!,1432771200,{'Format:': ' Paperback'},garminsuperformat paperback,delorm new york state atla amp gazett
3,3,"['Sports & Outdoors', 'Outdoor Recreation', 'C...","[""Rely on delorme ATLAS & gazetteer paper maps...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","['>#84,484 in Office Products (See top 100)', ...",[],...,$21.96,0899333257,5.0,True,delorm atla us state theyr amaz detail accur e...,Necessary for off grid camping,1505260800,{'Format:': ' Paperback'},garminnecessari grid campingformat paperback,garmin delorm atla amp gazett paper map arizona
4,4,"['Sports & Outdoors', 'Outdoor Recreation', 'C...","[""Rely on delorme ATLAS & gazetteer paper maps...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","['>#84,484 in Office Products (See top 100)', ...",[],...,$21.96,0899333257,5.0,False,first delorm atla gazett ever purchas year ago...,Don't explore without it,1470614400,,garmindont explor without,garmin delorm atla amp gazett paper map arizona


In [14]:
df_all_15 = df_all_15[~(df_all_15['title_nlp'].isna())]
df_all_15 = df_all_15[~(df_all_15['item_text'].isna())]
df_all_15 = df_all_15[~(df_all_15['reviewText'].isna())]
                         

In [15]:
df_all_15.shape

(369155, 21)

In [16]:
df_items = df_all_15.groupby(['asin', 'title','title_nlp','image_x','item_text'])['reviewText'].apply(lambda x: ' '.join(x)).reset_index()

In [17]:
df_items

,asin,title,title_nlp,image_x,item_text,reviewText
0,0899332757,Delorme New York State Atlas &amp; Gazetteer,delorm new york state atla amp gazett,['https://images-na.ssl-images-amazon.com/imag...,garminfour starsformat paperback,wonder comprehens atla littl larg
1,0899332757,Delorme New York State Atlas &amp; Gazetteer,delorm new york state atla amp gazett,['https://images-na.ssl-images-amazon.com/imag...,garmingreat explor rural part stateformat pape...,your interest explor area new york beyond metr...
2,0899332757,Delorme New York State Atlas &amp; Gazetteer,delorm new york state atla amp gazett,['https://images-na.ssl-images-amazon.com/imag...,garminsuperformat paperback,great product fast deliveri need rv travel
3,0899333257,Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,garmin delorm atla amp gazett paper map arizona,['https://images-na.ssl-images-amazon.com/imag...,garmina state wide open spacesformat paperback,delorm arizona atla gazett excel resourc trip ...
4,0899333257,Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,garmin delorm atla amp gazett paper map arizona,['https://images-na.ssl-images-amazon.com/imag...,garmindont explor without,first delorm atla gazett ever purchas year ago...
...,...,...,...,...,...,...
316345,B01HJDHNX2,Cycling Computers- Wireless Bike Speedometer a...,cycl comput wireless bike speedomet odometerwa...,['https://images-na.ssl-images-amazon.com/imag...,great tool peopl realli like ride bike,nbspa friend mine go big bike trip week excit ...
316346,B01HJDHNX2,Cycling Computers- Wireless Bike Speedometer a...,cycl comput wireless bike speedomet odometerwa...,['https://images-na.ssl-images-amazon.com/imag...,lot good inform fingertip,im pro cycler could consid causal rider love g...
316347,B01HJDHNX2,Cycling Computers- Wireless Bike Speedometer a...,cycl comput wireless bike speedomet odometerwa...,['https://images-na.ssl-images-amazon.com/imag...,nice cycl comput,perfect speedomet anyon look larg display inex...
316348,B01HJDZ34I,Columbia Bora booney,columbia bora booney,['https://images-na.ssl-images-amazon.com/imag...,columbiadec hat work well normal duti needssiz...,decent hat work well normal duti need still wa...


In [12]:
s3.list_buckets()['Buckets']

[{'Name': 'recommender-system-amazon',
  'CreationDate': datetime.datetime(2020, 9, 14, 18, 7, 18, tzinfo=tzlocal())}]

In [13]:
bucket = 'recommender-system-amazon' 
csv_buffer = StringIO()
df_items.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df_items.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'CP1X3T2XFX0H2XCT',
  'HostId': 'i6psuVPWJXKLjUXR045uzCdhMnfqw55zVs/t3Bzs35L5XjVbnJ/a9LrjXNDk4Qfp+NtshfjCL4I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'i6psuVPWJXKLjUXR045uzCdhMnfqw55zVs/t3Bzs35L5XjVbnJ/a9LrjXNDk4Qfp+NtshfjCL4I=',
   'x-amz-request-id': 'CP1X3T2XFX0H2XCT',
   'date': 'Wed, 16 Sep 2020 17:10:48 GMT',
   'etag': '"4872fcad402193e555bf3e2b752bf079"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4872fcad402193e555bf3e2b752bf079"'}

## Process text and title_nlp separately

In [22]:
vectorizer_item_text, df_item_text = nlp_process(df_items['item_text'],max_features = None)

In [23]:
df_item_text

,absorb,accessori,accur,ace,activ,actual,acu,add,addit,ade,...,xsmall,xxlarg,xxxlarg,year,yellow,yoga,youth,zero,zip,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
vectorizer_title, df_title = nlp_process(df_items['title_nlp'],max_features = None)

In [25]:
df_title

,aaa,ab,abras,absorb,access,accessori,ace,action,activ,adapt,...,xtreme,yard,year,yellow,yoga,youth,zero,zip,zipper,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
vectorizer_review, df_review = nlp_process(df_items['reviewText'],max_features = None)

In [27]:
df_review

,aa,aaa,ab,abil,abl,absolut,absorb,abus,accept,access,...,younger,youth,youtub,youv,yr,zero,zip,zipper,zone,zoom
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.102625,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316345,0.0,0.0,0.0,0.0,0.000000,0.099837,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
316346,0.0,0.0,0.0,0.0,0.159728,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.10887,0.0,0.0,0.0
316347,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
316348,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


#### add daraftames to s3

In [43]:
df_text.to_csv(csv_buffer)
s3_resource.Object(bucket, 'df_text.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '5AC3EF2111EFC682',
  'HostId': 'Etxo8aEwhTbv3RtEoAdmgHuvkGoicyD0sauY9JB1+Qplz43W1uPYmmPrHfZv886kyqS8BGGNiIA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Etxo8aEwhTbv3RtEoAdmgHuvkGoicyD0sauY9JB1+Qplz43W1uPYmmPrHfZv886kyqS8BGGNiIA=',
   'x-amz-request-id': '5AC3EF2111EFC682',
   'date': 'Tue, 15 Sep 2020 22:23:02 GMT',
   'etag': '"543ed8010cef492b570033d276c42157"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"543ed8010cef492b570033d276c42157"'}

In [44]:
df_title.to_csv(csv_buffer)
s3_resource.Object(bucket, 'df_title.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '603B2578A56B7906',
  'HostId': 'M8ivFyAOgn4CGKj96f/Yd0FArJO+2xZzmjqOF/8NHgNrWqLlFYdzqfBBi027vY98dM/lEyvq658=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'M8ivFyAOgn4CGKj96f/Yd0FArJO+2xZzmjqOF/8NHgNrWqLlFYdzqfBBi027vY98dM/lEyvq658=',
   'x-amz-request-id': '603B2578A56B7906',
   'date': 'Tue, 15 Sep 2020 22:23:32 GMT',
   'etag': '"a21e770cfc2d06614b0058846c8f82f6"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"a21e770cfc2d06614b0058846c8f82f6"'}

### Combine tile, context, and reviews with a specific weight

In [28]:
def weighted_vector(content_matrix, title_matrix, alpha):
    content_matrix = content_matrix * alpha
    title_matrix = title_matrix * (1-alpha)
    return content_matrix.add(title_matrix, fill_value=0)

In [ ]:
weighted_vector = weighted_vector(df_item_text, df_title, .8)


In [29]:
weighted_vector.shape

(56351, 2516)

In [34]:
df_items['title'][100]

'PEET Shoe Dryer Original M97-OL Shoe Dryer (Discontinued by MFG 2009)'

In [31]:
news_recommender = ContentRecommender()
news_recommender.fit(weighted_vector.values, titles = df_items['title'])

,0,1,2,3,4,5,6,7,8,9,...,56341,56342,56343,56344,56345,56346,56347,56348,56349,56350
title,,,,,,,,,,,,,,,,,,,,,
Delorme New York State Atlas &amp; Gazetteer,1.000000,0.027495,0.000000,0.000000,0.000000,0.000000,0.021078,0.000000,0.000000,0.000000,...,0.008367,0.000000,0.0,0.000000,0.005441,0.000000,0.007543,0.000000,0.000000,0.000000
"Garmin DeLorme Atlas &amp; Gazetteer Paper Maps- Arizona, AA-000005-000",0.027495,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000666,0.000000,0.0,0.000000,0.000433,0.000000,0.000600,0.000000,0.000000,0.000000
BenchMaster Pocket Guide - Fly Fishing - Fishing,0.000000,0.000000,1.000000,0.000000,0.019962,0.006680,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.066368,0.000000,0.025820,0.000000,0.000000
Spooner Boards Freestyle - Green,0.000000,0.000000,0.000000,1.000000,0.025442,0.011276,0.000000,0.000000,0.000000,0.293637,...,0.000000,0.000000,0.0,0.000000,0.011681,0.000000,0.016131,0.027239,0.000000,0.000000
Black Mountain Products Single Resistance Band - Door Anchor and Starter Guide Included,0.000000,0.000000,0.019962,0.025442,1.000000,0.684941,0.149299,0.000000,0.157911,0.000000,...,0.000000,0.067953,0.0,0.000000,0.025163,0.051753,0.009856,0.055259,0.116562,0.020251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MEILAIER UV Fishing Mask Moisture Wicking Headband Seamless Bandana COOLMAX Microfiber for Cycling Running Hunting Motorcycle Head Wear,0.000000,0.000000,0.066368,0.000000,0.051753,0.012481,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000335,0.054683,0.000000,0.000000
"Soft Cooling Towel for Instant Relief, 48inch Extra Long Natural Cool Cloth Scarf, Cold Neck Wrap Chilly Towels for Sports, Workout, Fitness, Gym, Yoga, Pilates, Travel, Camping Outdoor Running &amp; More",0.007543,0.000600,0.000000,0.016131,0.009856,0.013985,0.000000,0.002626,0.000000,0.002363,...,0.019673,0.014231,0.0,0.000630,0.007395,0.000335,1.000000,0.012739,0.009764,0.019212
Cycling Computers- Wireless Bike Speedometer and Odometer-Waterproof Digital Bike Computer -Large HD LCD Screen Display Bicycle Speedometer - High Accuracy for Powerful Magnet -Black,0.000000,0.000000,0.025820,0.027239,0.055259,0.035672,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.013680,0.0,0.000594,0.082826,0.054683,0.012739,1.000000,0.002678,0.000000


In [36]:
news_recommender.get_recommendations(item='PEET Shoe Dryer Original M97-OL Shoe Dryer (Discontinued by MFG 2009)')

array(["Invicta Men's 5513 Subaqua Collection Chronograph Watch",
       'Gaiam Restore Total Body Foam Roller with DVD, 36-Inch',
       'PUBLIC Bikes Trieste Coffee Cup Holder, Silver',
       'Taylor Made Products Boat Seat Cover (Back to Back Lounge Seat)',
       '150 lb Black Metal Hunting Crossbow Archery Bow + 4x20 Scope +12 Bolts 180 80 50'],
      dtype=object)

In [77]:
!source activate tensorflow_p36

/bin/sh: 1: source: not found
